# 📊 Nonprofit Data Analysis

This notebook provides comprehensive analysis of donor-advised funds, private foundations, and grants data collected using the consolidated `data_collector.py` script.

## 🎯 Analysis Overview

This analysis includes:
- **Geographic distribution** of foundations and donor-advised funds
- **Financial analysis** of foundation assets and revenues
- **Grant patterns** and funding trends
- **Research insights** for grant seeking organizations

## 📁 Data Sources

The data analyzed here comes from:
- **ProPublica Nonprofit Explorer API**: Organizational data for 2M+ tax-exempt organizations
- **IRS 990 Forms**: Grants data extracted from Schedule I (when available)
- **Research Resources**: Prioritized lists and collection templates for systematic grants research

Run the consolidated data collector first:
```bash
python data_collector.py --include-grants
```

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

# Configure pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("📊 Nonprofit Data Analysis Toolkit")
print("=" * 50)
print("✅ Libraries imported successfully")
print(f"📅 Analysis run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Check if data directory exists
data_dir = Path('data')
if data_dir.exists():
    print(f"📁 Data directory found: {data_dir}")
    files = list(data_dir.glob('*.csv'))
    print(f"📊 Available data files: {len(files)}")
    for file in files:
        print(f"   • {file.name}")
else:
    print("⚠️  Data directory not found. Run data_collector.py first.")

## 1. Load Data

Let's load the most recent datasets collected by the data collection script.

In [ ]:
# Function to load the most recent datasets
def load_latest_data():
    """Load the most recent data files from the data directory"""
    
    # Load all available datasets
    data_files = {
        'donor_advised_funds': 'data/donor_advised_funds.csv',
        'private_foundations': 'data/private_foundations.csv',
        'grants': 'data/grants.csv',
        'research_plan': 'data/grants_research_plan.csv',
        'collection_template': 'data/grants_collection_template.csv'
    }

    # Load datasets that exist
    datasets = {}
    for name, filepath in data_files.items():
        file_path = Path(filepath)
        if file_path.exists():
            try:
                datasets[name] = pd.read_csv(file_path)
                print(f"✅ Loaded {name}: {len(datasets[name])} records")
            except Exception as e:
                print(f"❌ Error loading {name}: {e}")
        else:
            print(f"⚠️  File not found: {filepath}")

    # Load collection summary if available
    summary_file = Path('data/collection_summary.json')
    collection_summary = None
    if summary_file.exists():
        try:
            with open(summary_file, 'r') as f:
                collection_summary = json.load(f)
            print(f"✅ Loaded collection summary: {collection_summary['total_runs']} runs")
            print(f"📅 Last updated: {collection_summary['last_updated']}")
        except Exception as e:
            print(f"❌ Error loading collection summary: {e}")

    print("\n📊 Dataset Overview:")
    print("-" * 30)
    for name, df in datasets.items():
        if isinstance(df, pd.DataFrame):
            print(f"{name}: {len(df)} records, {len(df.columns)} columns")
    
    return datasets

# Load the data
datasets = load_latest_data()

donor_funds_df = datasets.get('donor_advised_funds', pd.DataFrame())
foundations_df = datasets.get('private_foundations', pd.DataFrame())
grants_df = datasets.get('grants', pd.DataFrame())

In [ ]:
# Quick data overview
if donor_funds_df is not None:
    print("=" * 60)
    print("📊 DATASET OVERVIEW")
    print("=" * 60)
    
    print("\n🏦 DONOR-ADVISED FUNDS")
    print(f"Total organizations: {len(donor_funds_df)}")
    if 'revenue_amount' in donor_funds_df.columns:
        print(f"Average revenue: ${donor_funds_df['revenue_amount'].mean():,.0f}")
        print(f"Total revenue: ${donor_funds_df['revenue_amount'].sum():,.0f}")
    
    print("\n🏛️ PRIVATE FOUNDATIONS") 
    print(f"Total organizations: {len(foundations_df)}")
    if 'asset_amount' in foundations_df.columns:
        print(f"Average assets: ${foundations_df['asset_amount'].mean():,.0f}")
        print(f"Total assets: ${foundations_df['asset_amount'].sum():,.0f}")
    
    if not grants_df.empty:
        print("\n💰 GRANTS DATA")
        print(f"Total grant records: {len(grants_df)}")
        print(f"Total grant amount: ${grants_df['amount'].sum():,.2f}")
        print(f"Average grant: ${grants_df['amount'].mean():,.2f}")
        print(f"Median grant: ${grants_df['amount'].median():,.2f}")
    
    print("\n📍 GEOGRAPHIC DISTRIBUTION")
    all_orgs = pd.concat([donor_funds_df, foundations_df], ignore_index=True)
    if 'state' in all_orgs.columns:
        top_states = all_orgs['state'].value_counts().head()
        for state, count in top_states.items():
            print(f"   {state}: {count} organizations")
    
    print("=" * 60)

# Combine all organizational data for comprehensive analysis
organizational_data = []

if 'donor_advised_funds' in datasets:
    donor_funds = datasets['donor_advised_funds'].copy()
    donor_funds['data_source'] = 'Donor Advised Funds'
    organizational_data.append(donor_funds)

if 'private_foundations' in datasets:
    foundations = datasets['private_foundations'].copy()
    foundations['data_source'] = 'Private Foundations'
    organizational_data.append(foundations)

if organizational_data:
    # Combine all organizational data
    all_orgs = pd.concat(organizational_data, ignore_index=True)
    
    print("🏢 Organizational Data Summary")
    print("=" * 40)
    print(f"Total organizations: {len(all_orgs)}")
    print(f"Donor-advised funds: {len(all_orgs[all_orgs['data_source'] == 'Donor Advised Funds'])}")
    print(f"Private foundations: {len(all_orgs[all_orgs['data_source'] == 'Private Foundations'])}")
    
    # Geographic distribution
    print(f"\n🌍 Geographic Distribution (Top 10 States):")
    state_counts = all_orgs['state'].value_counts().head(10)
    for state, count in state_counts.items():
        print(f"   {state}: {count}")
    
    # Financial summary (if revenue/assets data available)
    if 'revenue' in all_orgs.columns:
        revenue_data = all_orgs[all_orgs['revenue'] > 0]
        if not revenue_data.empty:
            print(f"\n💰 Financial Summary:")
            print(f"   Organizations with revenue data: {len(revenue_data)}")
            print(f"   Total revenue: ${revenue_data['revenue'].sum():,.0f}")
            print(f"   Average revenue: ${revenue_data['revenue'].mean():,.0f}")
            print(f"   Median revenue: ${revenue_data['revenue'].median():,.0f}")
    
    # Collection timing analysis
    if 'collection_date' in all_orgs.columns:
        print(f"\n📅 Data Collection Info:")
        collection_dates = all_orgs['collection_date'].value_counts()
        for date, count in collection_dates.items():
            print(f"   {date}: {count} organizations")
else:
    print("⚠️  No organizational data available. Run data_collector.py first.")

## 2. Configure Your Nonprofit Focus Area

**🎯 CUSTOMIZE THIS SECTION FOR YOUR ORGANIZATION**

Choose keywords that match your nonprofit's mission and programs. This will help identify funders who support similar causes.

In [ ]:
# 🎯 FOCUS AREA CONFIGURATIONS
# Choose the one that matches your nonprofit or create your own

FOCUS_AREAS = {
    'education': {
        'keywords': ['education', 'school', 'student', 'learning', 'teacher', 'classroom', 
                    'literacy', 'scholarship', 'academic', 'curriculum', 'STEM', 'university', 'college'],
        'description': 'Educational institutions and programs'
    },
    'health': {
        'keywords': ['health', 'medical', 'healthcare', 'hospital', 'clinic', 'patient', 
                    'disease', 'treatment', 'research', 'mental health', 'wellness', 'therapy'],
        'description': 'Health and medical services'
    },
    'arts': {
        'keywords': ['arts', 'culture', 'music', 'theater', 'theatre', 'dance', 'visual arts', 
                    'museum', 'gallery', 'performance', 'artist', 'creative', 'cultural'],
        'description': 'Arts and cultural organizations'
    },
    'environment': {
        'keywords': ['environment', 'environmental', 'conservation', 'sustainability', 'climate', 
                    'wildlife', 'renewable', 'green', 'ecosystem', 'pollution', 'nature'],
        'description': 'Environmental and conservation efforts'
    },
    'social_services': {
        'keywords': ['social services', 'community', 'homeless', 'housing', 'poverty', 'food bank', 
                    'shelter', 'human services', 'family services', 'youth', 'seniors'],
        'description': 'Social and community services'
    }
}

# 📝 SELECT YOUR FOCUS AREA
# Change this to match your nonprofit's mission
CHOSEN_FOCUS = 'education'  # ← CHANGE THIS

# Get the selected focus area
if CHOSEN_FOCUS in FOCUS_AREAS:
    focus_config = FOCUS_AREAS[CHOSEN_FOCUS]
    chosen_keywords = focus_config['keywords']
    focus_description = focus_config['description']
    
    print(f"🎯 Selected focus area: {CHOSEN_FOCUS.title()}")
    print(f"📝 Description: {focus_description}")
    print(f"🔑 Keywords: {', '.join(chosen_keywords[:10])}...")  # Show first 10
else:
    print("❌ Invalid focus area selected")
    
# You can also create custom keywords
CUSTOM_KEYWORDS = []  # Add your own keywords here if needed
if CUSTOM_KEYWORDS:
    chosen_keywords.extend(CUSTOM_KEYWORDS)
    print(f"✅ Added {len(CUSTOM_KEYWORDS)} custom keywords")

# 📊 GEOGRAPHIC ANALYSIS
# This section analyzes the geographic distribution of organizations

if 'all_orgs' in locals() and not all_orgs.empty:
    # Create geographic analysis visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('🌍 Geographic Distribution Analysis', fontsize=16, fontweight='bold')
    
    # 1. Top 15 states by total organizations
    ax1 = axes[0, 0]
    top_states = all_orgs['state'].value_counts().head(15)
    top_states.plot(kind='bar', ax=ax1, color='skyblue')
    ax1.set_title('Organizations by State (Top 15)')
    ax1.set_xlabel('State')
    ax1.set_ylabel('Number of Organizations')
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. Organization type distribution
    ax2 = axes[0, 1]
    type_counts = all_orgs['data_source'].value_counts()
    ax2.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Organization Type Distribution')
    
    # 3. Revenue distribution by state (top 10 states)
    ax3 = axes[1, 0]
    if 'revenue' in all_orgs.columns:
        revenue_by_state = all_orgs.groupby('state')['revenue'].sum().sort_values(ascending=False).head(10)
        revenue_by_state.plot(kind='bar', ax=ax3, color='lightgreen')
        ax3.set_title('Total Revenue by State (Top 10)')
        ax3.set_xlabel('State')
        ax3.set_ylabel('Total Revenue ($)')
        ax3.tick_params(axis='x', rotation=45)
        
        # Format y-axis to show values in millions/billions
        ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M' if x < 1e9 else f'${x/1e9:.1f}B'))
    else:
        ax3.text(0.5, 0.5, 'Revenue data not available', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Revenue Analysis - Data Not Available')
    
    # 4. Comparison: Donor Funds vs Foundations by state
    ax4 = axes[1, 1]
    state_type_crosstab = pd.crosstab(all_orgs['state'], all_orgs['data_source'])
    top_states_for_comparison = all_orgs['state'].value_counts().head(10).index
    state_type_subset = state_type_crosstab.loc[top_states_for_comparison]
    state_type_subset.plot(kind='bar', ax=ax4, stacked=True, color=['lightcoral', 'lightblue'])
    ax4.set_title('Organization Types by State (Top 10)')
    ax4.set_xlabel('State')
    ax4.set_ylabel('Number of Organizations')
    ax4.tick_params(axis='x', rotation=45)
    ax4.legend(title='Organization Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed state analysis
    print("\n📊 Detailed State Analysis:")
    print("-" * 50)
    state_summary = all_orgs.groupby('state').agg({
        'ein': 'count',
        'revenue': ['sum', 'mean'] if 'revenue' in all_orgs.columns else 'count',
        'data_source': lambda x: x.value_counts().to_dict()
    }).round(0)
    
    if 'revenue' in all_orgs.columns:
        state_summary.columns = ['Total_Orgs', 'Total_Revenue', 'Avg_Revenue', 'Org_Types']
    else:
        state_summary.columns = ['Total_Orgs', 'Org_Types']
    
    # Show top 10 states
    top_10_states = state_summary.nlargest(10, 'Total_Orgs')
    print(top_10_states)
    
else:
    print("⚠️  No organizational data available for geographic analysis.")

## 3. Grant Analysis Functions

These functions help us analyze the grants data to find relevant funders.

In [ ]:
def find_funders_by_keywords(grants_df, keywords, min_grant_amount=5000):
    """Find funders that have made grants related to specific keywords"""
    if grants_df.empty:
        return pd.DataFrame()
    
    # Create keyword pattern
    keyword_pattern = '|'.join(keywords)
    
    # Search in purpose field
    if 'purpose' in grants_df.columns:
        mask = grants_df['purpose'].str.contains(keyword_pattern, case=False, na=False)
        matching_grants = grants_df[mask]
    else:
        matching_grants = pd.DataFrame()
    
    if matching_grants.empty:
        return pd.DataFrame()
    
    # Filter by minimum amount
    matching_grants = matching_grants[matching_grants['amount'] >= min_grant_amount]
    
    # Summarize by grantor
    summary = matching_grants.groupby(['grantor_ein', 'grantor_name', 'grantor_type']).agg({
        'amount': ['sum', 'count', 'mean'],
        'filing_year': 'max'
    }).round(2)
    
    summary.columns = ['total_grants', 'grant_count', 'avg_grant', 'latest_filing']
    summary = summary.reset_index()
    summary = summary.sort_values('total_grants', ascending=False)
    
    return summary

def find_funders_by_recipient_type(grants_df, recipient_keywords, min_grant_amount=5000):
    """Find funders that give to specific types of organizations"""
    if grants_df.empty:
        return pd.DataFrame()
    
    # Create keyword pattern
    keyword_pattern = '|'.join(recipient_keywords)
    
    # Search in recipient names
    if 'recipient_name' in grants_df.columns:
        mask = grants_df['recipient_name'].str.contains(keyword_pattern, case=False, na=False)
        matching_grants = grants_df[mask]
    else:
        matching_grants = pd.DataFrame()
    
    if matching_grants.empty:
        return pd.DataFrame()
    
    # Filter by minimum amount
    matching_grants = matching_grants[matching_grants['amount'] >= min_grant_amount]
    
    # Summarize by grantor
    summary = matching_grants.groupby(['grantor_ein', 'grantor_name', 'grantor_type']).agg({
        'amount': ['sum', 'count', 'mean'],
        'filing_year': 'max'
    }).round(2)
    
    summary.columns = ['total_grants', 'grant_count', 'avg_grant', 'latest_filing']
    summary = summary.reset_index()
    summary = summary.sort_values('total_grants', ascending=False)
    
    return summary

def analyze_funder_patterns(grants_df, grantor_ein):
    """Analyze the grant-making patterns of a specific funder"""
    grantor_grants = grants_df[grants_df['grantor_ein'] == grantor_ein]
    
    if grantor_grants.empty:
        return {}
    
    analysis = {
        'grantor_name': grantor_grants['grantor_name'].iloc[0],
        'total_grants': len(grantor_grants),
        'total_amount': grantor_grants['amount'].sum(),
        'avg_grant': grantor_grants['amount'].mean(),
        'median_grant': grantor_grants['amount'].median(),
        'grant_range': (grantor_grants['amount'].min(), grantor_grants['amount'].max()),
        'years_active': grantor_grants['filing_year'].nunique(),
        'year_range': (grantor_grants['filing_year'].min(), grantor_grants['filing_year'].max()),
    }
    
    # Top recipients
    if 'recipient_name' in grantor_grants.columns:
        top_recipients = grantor_grants.groupby('recipient_name')['amount'].sum().sort_values(ascending=False).head(5)
        analysis['top_recipients'] = top_recipients.to_dict()
    
    return analysis

print("✅ Analysis functions defined!")

# Grants Data Analysis
print("💰 Grants Data Analysis")
print("=" * 40)

if 'grants' in datasets and not datasets['grants'].empty:
    grants_df = datasets['grants']
    
    print(f"Total grant records: {len(grants_df)}")
    
    # Basic grants statistics
    if 'amount' in grants_df.columns:
        grant_amounts = grants_df[grants_df['amount'] > 0]['amount']
        if not grant_amounts.empty:
            print(f"Total grant amount: ${grant_amounts.sum():,.2f}")
            print(f"Average grant size: ${grant_amounts.mean():,.2f}")
            print(f"Median grant size: ${grant_amounts.median():,.2f}")
            print(f"Largest grant: ${grant_amounts.max():,.2f}")
            print(f"Smallest grant: ${grant_amounts.min():,.2f}")
            
            # Grant size distribution
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Histogram of grant amounts
            ax1 = axes[0]
            grant_amounts.hist(bins=50, ax=ax1, alpha=0.7, color='lightgreen')
            ax1.set_title('Distribution of Grant Amounts')
            ax1.set_xlabel('Grant Amount ($)')
            ax1.set_ylabel('Frequency')
            ax1.axvline(grant_amounts.mean(), color='red', linestyle='--', label=f'Mean: ${grant_amounts.mean():,.0f}')
            ax1.axvline(grant_amounts.median(), color='blue', linestyle='--', label=f'Median: ${grant_amounts.median():,.0f}')
            ax1.legend()
            
            # Top grantors by total giving
            ax2 = axes[1]
            if 'grantor_name' in grants_df.columns:
                top_grantors = grants_df.groupby('grantor_name')['amount'].sum().sort_values(ascending=False).head(10)
                top_grantors.plot(kind='barh', ax=ax2, color='lightcoral')
                ax2.set_title('Top 10 Grantors by Total Giving')
                ax2.set_xlabel('Total Grant Amount ($)')
            else:
                ax2.text(0.5, 0.5, 'Grantor data not available', ha='center', va='center', transform=ax2.transAxes)
            
            plt.tight_layout()
            plt.show()
    
    # Grants by year analysis
    if 'grant_year' in grants_df.columns:
        yearly_grants = grants_df.groupby('grant_year').agg({
            'amount': ['count', 'sum', 'mean']
        }).round(2)
        yearly_grants.columns = ['Grant_Count', 'Total_Amount', 'Average_Amount']
        
        print(f"\n📅 Grants by Year:")
        print(yearly_grants)
        
        # Plot yearly trends
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        ax1 = axes[0]
        yearly_grants['Grant_Count'].plot(kind='bar', ax=ax1, color='skyblue')
        ax1.set_title('Number of Grants by Year')
        ax1.set_xlabel('Year')
        ax1.set_ylabel('Number of Grants')
        
        ax2 = axes[1]
        yearly_grants['Total_Amount'].plot(kind='bar', ax=ax2, color='lightgreen')
        ax2.set_title('Total Grant Amount by Year')
        ax2.set_xlabel('Year')
        ax2.set_ylabel('Total Amount ($)')
        
        plt.tight_layout()
        plt.show()

else:
    print("📋 No grants data available yet.")
    print("\n🎯 To collect grants data:")
    print("   1. Run: python data_collector.py --include-grants")
    print("   2. Use the generated grants_research_plan.csv for systematic research")
    print("   3. Record findings in grants_collection_template.csv")
    print("   4. Consider subscribing to Candid Foundation Directory")
    
    # Show research plan if available
    if 'research_plan' in datasets:
        research_plan = datasets['research_plan']
        print(f"\n📊 Research Plan Available:")
        print(f"   • {len(research_plan)} foundations to research")
        print(f"   • Research URLs and strategies provided")
        print(f"   • Status tracking columns for progress monitoring")
        
        # Show top research targets
        print(f"\n🎯 Top Research Targets:")
        top_targets = research_plan.head(10)[['name', 'city', 'state', 'research_priority']]
        print(top_targets.to_string(index=False))

## 4. Data Visualization

Let's create visualizations to understand our potential funder landscape.

In [ ]:
# Create comprehensive visualizations
if donor_funds_df is not None and not donor_funds_df.empty:
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('ProPublica Donor Data Analysis Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Geographic distribution of all organizations
    all_orgs = pd.concat([donor_funds_df, foundations_df], ignore_index=True)
    if 'state' in all_orgs.columns:
        top_states = all_orgs['state'].value_counts().head(10)
        axes[0, 0].bar(top_states.index, top_states.values)
        axes[0, 0].set_title('Organizations by State')
        axes[0, 0].set_xlabel('State')
        axes[0, 0].set_ylabel('Count')
        axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Organization type distribution
    type_counts = all_orgs['organization_type'].value_counts()
    axes[0, 1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
    axes[0, 1].set_title('Organization Types')
    
    # 3. Revenue distribution (donor-advised funds)
    if 'revenue_amount' in donor_funds_df.columns:
        revenue_data = donor_funds_df['revenue_amount'].dropna()
        if not revenue_data.empty:
            axes[0, 2].hist(revenue_data, bins=20, alpha=0.7, edgecolor='black')
            axes[0, 2].set_title('Revenue Distribution - Donor-Advised Funds')
            axes[0, 2].set_xlabel('Revenue ($)')
            axes[0, 2].set_ylabel('Frequency')
            axes[0, 2].set_xscale('log')
    
    # 4. Asset distribution (private foundations)
    if 'asset_amount' in foundations_df.columns:
        asset_data = foundations_df['asset_amount'].dropna()
        if not asset_data.empty:
            axes[1, 0].hist(asset_data, bins=20, alpha=0.7, color='orange', edgecolor='black')
            axes[1, 0].set_title('Asset Distribution - Private Foundations')
            axes[1, 0].set_xlabel('Assets ($)')
            axes[1, 0].set_ylabel('Frequency')
            axes[1, 0].set_xscale('log')
    
    # 5. Grant amount distribution
    if not grants_df.empty:
        grant_amounts = grants_df['amount'].dropna()
        if not grant_amounts.empty:
            axes[1, 1].hist(grant_amounts, bins=30, alpha=0.7, color='green', edgecolor='black')
            axes[1, 1].set_title('Grant Amount Distribution')
            axes[1, 1].set_xlabel('Grant Amount ($)')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].set_xscale('log')
    
    # 6. Top grantors by total amount
    if not grants_df.empty:
        top_grantors = grants_df.groupby('grantor_name')['amount'].sum().sort_values(ascending=False).head(8)
        if not top_grantors.empty:
            y_pos = np.arange(len(top_grantors))
            axes[1, 2].barh(y_pos, top_grantors.values)
            axes[1, 2].set_yticks(y_pos)
            axes[1, 2].set_yticklabels([name[:25] + '...' if len(name) > 25 else name 
                                       for name in top_grantors.index])
            axes[1, 2].set_title('Top Grantors by Total Amount')
            axes[1, 2].set_xlabel('Total Grant Amount ($)')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No data available for visualization")
    print("💡 Run 'python collect_data.py' first to collect data")

# Financial Analysis of Organizations
print("💰 Financial Analysis")
print("=" * 40)

if 'all_orgs' in locals() and not all_orgs.empty:
    # Filter organizations with financial data
    financial_orgs = all_orgs[(all_orgs['revenue'] > 0) | (all_orgs['assets'] > 0)]
    
    if not financial_orgs.empty:
        print(f"Organizations with financial data: {len(financial_orgs)} / {len(all_orgs)}")
        
        # Revenue analysis
        revenue_orgs = financial_orgs[financial_orgs['revenue'] > 0]
        if not revenue_orgs.empty:
            print(f"\n📊 Revenue Analysis ({len(revenue_orgs)} organizations):")
            print(f"   Total revenue: ${revenue_orgs['revenue'].sum():,.0f}")
            print(f"   Average revenue: ${revenue_orgs['revenue'].mean():,.0f}")
            print(f"   Median revenue: ${revenue_orgs['revenue'].median():,.0f}")
            print(f"   Top revenue: ${revenue_orgs['revenue'].max():,.0f}")
        
        # Assets analysis
        asset_orgs = financial_orgs[financial_orgs['assets'] > 0]
        if not asset_orgs.empty:
            print(f"\n🏦 Assets Analysis ({len(asset_orgs)} organizations):")
            print(f"   Total assets: ${asset_orgs['assets'].sum():,.0f}")
            print(f"   Average assets: ${asset_orgs['assets'].mean():,.0f}")
            print(f"   Median assets: ${asset_orgs['assets'].median():,.0f}")
            print(f"   Top assets: ${asset_orgs['assets'].max():,.0f}")
        
        # Create financial visualizations
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('💰 Financial Analysis of Organizations', fontsize=16, fontweight='bold')
        
        # 1. Revenue distribution by organization type
        ax1 = axes[0, 0]
        if not revenue_orgs.empty:
            revenue_by_type = revenue_orgs.groupby('data_source')['revenue'].sum()
            ax1.pie(revenue_by_type.values, labels=revenue_by_type.index, autopct='%1.1f%%', startangle=90)
            ax1.set_title('Total Revenue by Organization Type')
        else:
            ax1.text(0.5, 0.5, 'Revenue data not available', ha='center', va='center', transform=ax1.transAxes)
        
        # 2. Top 10 organizations by revenue
        ax2 = axes[0, 1]
        if not revenue_orgs.empty:
            top_revenue = revenue_orgs.nlargest(10, 'revenue')[['name', 'revenue']]
            # Truncate long names for display
            display_names = [name[:30] + '...' if len(name) > 30 else name for name in top_revenue['name']]
            ax2.barh(range(len(display_names)), top_revenue['revenue'], color='lightgreen')
            ax2.set_yticks(range(len(display_names)))
            ax2.set_yticklabels(display_names)
            ax2.set_title('Top 10 Organizations by Revenue')
            ax2.set_xlabel('Revenue ($)')
            # Format x-axis
            ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M' if x < 1e9 else f'${x/1e9:.1f}B'))
        else:
            ax2.text(0.5, 0.5, 'Revenue data not available', ha='center', va='center', transform=ax2.transAxes)
        
        # 3. Revenue vs Assets scatter plot
        ax3 = axes[1, 0]
        orgs_with_both = financial_orgs[(financial_orgs['revenue'] > 0) & (financial_orgs['assets'] > 0)]
        if not orgs_with_both.empty:
            colors = ['red' if org_type == 'Donor Advised Funds' else 'blue' for org_type in orgs_with_both['data_source']]
            ax3.scatter(orgs_with_both['assets'], orgs_with_both['revenue'], alpha=0.6, c=colors)
            ax3.set_xlabel('Assets ($)')
            ax3.set_ylabel('Revenue ($)')
            ax3.set_title('Revenue vs Assets')
            ax3.set_xscale('log')
            ax3.set_yscale('log')
            
            # Add legend
            import matplotlib.patches as mpatches
            red_patch = mpatches.Patch(color='red', label='Donor Advised Funds')
            blue_patch = mpatches.Patch(color='blue', label='Private Foundations')
            ax3.legend(handles=[red_patch, blue_patch])
        else:
            ax3.text(0.5, 0.5, 'Insufficient financial data', ha='center', va='center', transform=ax3.transAxes)
        
        # 4. Financial size categories
        ax4 = axes[1, 1]
        if not revenue_orgs.empty:
            # Create revenue size categories
            def categorize_revenue(revenue):
                if revenue < 100000:
                    return 'Small (<$100K)'
                elif revenue < 1000000:
                    return 'Medium ($100K-$1M)'
                elif revenue < 10000000:
                    return 'Large ($1M-$10M)'
                else:
                    return 'Very Large (>$10M)'
            
            revenue_orgs['size_category'] = revenue_orgs['revenue'].apply(categorize_revenue)
            size_counts = revenue_orgs['size_category'].value_counts()
            
            # Ensure proper ordering
            category_order = ['Small (<$100K)', 'Medium ($100K-$1M)', 'Large ($1M-$10M)', 'Very Large (>$10M)']
            size_counts = size_counts.reindex(category_order, fill_value=0)
            
            ax4.pie(size_counts.values, labels=size_counts.index, autopct='%1.1f%%', startangle=90)
            ax4.set_title('Organizations by Revenue Size')
        else:
            ax4.text(0.5, 0.5, 'Revenue data not available', ha='center', va='center', transform=ax4.transAxes)
        
        plt.tight_layout()
        plt.show()
        
        # Financial summary table
        if not revenue_orgs.empty:
            print(f"\n📋 Financial Summary by Organization Type:")
            financial_summary = revenue_orgs.groupby('data_source').agg({
                'revenue': ['count', 'sum', 'mean', 'median'],
                'assets': ['sum', 'mean', 'median'] if not asset_orgs.empty else 'count'
            }).round(0)
            
            print(financial_summary)
    
    else:
        print("⚠️  No financial data available in the current dataset.")
        print("   This may be due to API limitations or data collection scope.")

else:
    print("⚠️  No organizational data available for financial analysis.")

## 5. Find Relevant Funders

Now let's find funders that align with your chosen focus area.

In [ ]:
# Analyze grants data for relevant funders
if not grants_df.empty and 'chosen_keywords' in locals():
    
    print(f"🔍 Analyzing grants for {CHOSEN_FOCUS} funders...")
    print(f"🎯 Using keywords: {', '.join(chosen_keywords[:5])}...")
    
    # Find funders by grant keywords
    relevant_funders = find_funders_by_keywords(
        grants_df, 
        chosen_keywords, 
        min_grant_amount=5000
    )
    
    print(f"\n✅ Found {len(relevant_funders)} funders with {CHOSEN_FOCUS} grants")
    
    if not relevant_funders.empty:
        print(f"\n🏆 Top {CHOSEN_FOCUS.title()} Funders:")
        display(relevant_funders.head(10))
        
        # Analyze the top funder in detail
        if len(relevant_funders) > 0:
            top_funder_ein = relevant_funders.iloc[0]['grantor_ein']
            top_funder_analysis = analyze_funder_patterns(grants_df, top_funder_ein)
            
            print(f"\n🔬 Detailed Analysis of Top Funder:")
            print(f"   Name: {top_funder_analysis.get('grantor_name', 'Unknown')}")
            print(f"   Total grants made: {top_funder_analysis.get('total_grants', 0)}")
            print(f"   Total amount granted: ${top_funder_analysis.get('total_amount', 0):,.2f}")
            print(f"   Average grant: ${top_funder_analysis.get('avg_grant', 0):,.2f}")
            print(f"   Grant range: ${top_funder_analysis.get('grant_range', (0,0))[0]:,.0f} - ${top_funder_analysis.get('grant_range', (0,0))[1]:,.0f}")
            print(f"   Years active: {top_funder_analysis.get('years_active', 0)}")
            
            if 'top_recipients' in top_funder_analysis:
                print(f"\n   Top Recipients:")
                for recipient, amount in list(top_funder_analysis['top_recipients'].items())[:5]:
                    print(f"     • {recipient}: ${amount:,.2f}")
    
    else:
        print(f"❌ No funders found with {CHOSEN_FOCUS} grants in the dataset")
        print("💡 Try expanding keywords or checking the data collection")
    
    # Also search by recipient type
    recipient_keywords = FOCUS_AREAS[CHOSEN_FOCUS]['keywords'][:5]  # Use subset for recipient search
    recipient_funders = find_funders_by_recipient_type(
        grants_df,
        recipient_keywords,
        min_grant_amount=5000
    )
    
    if not recipient_funders.empty:
        print(f"\n🎯 Funders Supporting {CHOSEN_FOCUS.title()} Organizations:")
        display(recipient_funders.head(5))

else:
    if grants_df.empty:
        print("❌ No grants data available for analysis")
    else:
        print("⚠️ Focus area not configured properly")

# Research Insights and Grant-Seeking Recommendations
print("🎯 Research Insights & Grant-Seeking Recommendations")
print("=" * 60)

if 'all_orgs' in locals() and not all_orgs.empty:
    # Geographic targeting insights
    print("🌍 Geographic Targeting Insights:")
    print("-" * 40)
    
    top_states = all_orgs['state'].value_counts().head(10)
    print(f"🏆 Top 10 states for foundation concentration:")
    for i, (state, count) in enumerate(top_states.items(), 1):
        percentage = (count / len(all_orgs)) * 100
        print(f"   {i:2d}. {state}: {count:4d} organizations ({percentage:5.1f}%)")
    
    # Organization type insights
    print(f"\n🏢 Organization Type Distribution:")
    type_distribution = all_orgs['data_source'].value_counts()
    for org_type, count in type_distribution.items():
        percentage = (count / len(all_orgs)) * 100
        print(f"   • {org_type}: {count} ({percentage:.1f}%)")
    
    # Financial capacity insights (if available)
    if 'revenue' in all_orgs.columns:
        revenue_orgs = all_orgs[all_orgs['revenue'] > 0]
        if not revenue_orgs.empty:
            print(f"\n💰 Financial Capacity Insights:")
            print(f"   • Total sector revenue: ${revenue_orgs['revenue'].sum():,.0f}")
            print(f"   • Organizations with $1M+ revenue: {len(revenue_orgs[revenue_orgs['revenue'] >= 1000000])}")
            print(f"   • Organizations with $10M+ revenue: {len(revenue_orgs[revenue_orgs['revenue'] >= 10000000])}")
    
    # Research prioritization recommendations
    print(f"\n📊 Research Prioritization Strategy:")
    print("-" * 40)
    
    if 'research_plan' in datasets:
        research_plan = datasets['research_plan']
        print(f"✅ Research plan generated with {len(research_plan)} targets")
        print(f"📋 Recommended approach:")
        print(f"   1. Start with largest foundations in your state/region")
        print(f"   2. Focus on foundations with clear mission alignment")
        print(f"   3. Use provided research URLs for systematic investigation")
        print(f"   4. Track progress using status columns in research plan")
        
        # Geographic concentration analysis for targeting
        if not all_orgs.empty:
            state_analysis = all_orgs.groupby('state').size().sort_values(ascending=False)
            print(f"\n🎯 Geographic Targeting Recommendations:")
            print(f"   • Prioritize these high-concentration states:")
            for state in state_analysis.head(5).index:
                count = state_analysis[state]
                print(f"     - {state}: {count} foundations (high opportunity)")
    
    else:
        print(f"⚠️  Research plan not found. Run data_collector.py --include-grants to generate.")
    
    # Data collection recommendations
    print(f"\n📋 Next Steps for Comprehensive Grant Research:")
    print("-" * 50)
    print(f"1. 🎯 IMMEDIATE ACTIONS:")
    print(f"   • Review grants_research_plan.csv for prioritized targets")
    print(f"   • Start with foundations in your geographic area")
    print(f"   • Check foundation websites for published grants lists")
    print(f"   • Search local news for recent grant announcements")
    
    print(f"\n2. 📊 SYSTEMATIC RESEARCH:")
    print(f"   • Use grants_collection_template.csv to record findings")
    print(f"   • Focus on foundations with $1M+ in annual giving")
    print(f"   • Look for mission alignment with your organization")
    print(f"   • Track application deadlines and requirements")
    
    print(f"\n3. 🏆 ADVANCED STRATEGIES:")
    print(f"   • Subscribe to Candid Foundation Directory ($179+/month)")
    print(f"   • Download IRS 990 Schedule I forms for detailed grants data")
    print(f"   • Set up Google Alerts for foundation grant announcements")
    print(f"   • Network with other nonprofits for referrals and insights")
    
    print(f"\n4. 📈 DATA-DRIVEN APPROACH:")
    print(f"   • Analyze giving patterns and average grant sizes")
    print(f"   • Identify foundations that fund organizations like yours")
    print(f"   • Track success rates and adjust strategy accordingly")
    print(f"   • Build relationships before submitting proposals")

else:
    print("⚠️  No organizational data available for research insights.")
    print("   Run data_collector.py first to generate research recommendations.")

# Collection summary insights
if collection_summary:
    print(f"\n📅 Data Collection Summary:")
    print("-" * 30)
    print(f"   • Total collection runs: {collection_summary['total_runs']}")
    print(f"   • Last updated: {collection_summary['last_updated']}")
    
    latest_run = collection_summary['latest_run']
    print(f"   • Latest run collected:")
    print(f"     - {latest_run['donor_advised_funds_count']} donor-advised funds")
    print(f"     - {latest_run['private_foundations_count']} private foundations")
    print(f"     - {latest_run['total_grants']} grant records")
    
    if latest_run['total_grant_amount'] > 0:
        print(f"     - ${latest_run['total_grant_amount']:,.2f} in total grants")

print(f"\n✅ Analysis complete! Use this data to inform your grant-seeking strategy.")

## 6. Export Results

Let's create prospect lists and reports for fundraising outreach.

In [ ]:
# Create comprehensive reports for your fundraising efforts
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if donor_funds_df is not None and not donor_funds_df.empty:
    
    # 1. Create master prospect list
    all_orgs = pd.concat([donor_funds_df, foundations_df], ignore_index=True)
    
    # Select key columns for prospect list
    prospect_columns = ['name', 'city', 'state', 'ein', 'organization_type']
    if 'revenue_amount' in all_orgs.columns:
        prospect_columns.append('revenue_amount')
    if 'asset_amount' in all_orgs.columns:
        prospect_columns.append('asset_amount')
    
    prospect_list = all_orgs[prospect_columns].copy()
    
    # Sort by revenue/assets (whichever is available)
    if 'revenue_amount' in prospect_list.columns:
        prospect_list = prospect_list.sort_values('revenue_amount', ascending=False, na_last=True)
    elif 'asset_amount' in prospect_list.columns:
        prospect_list = prospect_list.sort_values('asset_amount', ascending=False, na_last=True)
    
    # Save prospect list
    prospect_filename = f'data/prospect_list_{CHOSEN_FOCUS}_{timestamp}.csv'
    prospect_list.to_csv(prospect_filename, index=False)
    print(f"📋 Saved prospect list: {prospect_filename}")
    
    # 2. Create focused funder report
    if not grants_df.empty and 'relevant_funders' in locals() and not relevant_funders.empty:
        
        # Enhanced funder report with contact info
        enhanced_funders = relevant_funders.copy()
        
        # Add organization details
        funder_details = []
        for _, funder in enhanced_funders.iterrows():
            ein = funder['grantor_ein']
            org_info = all_orgs[all_orgs['ein'] == ein]
            
            if not org_info.empty:
                org_data = org_info.iloc[0]
                funder_details.append({
                    'grantor_name': funder['grantor_name'],
                    'grantor_ein': ein,
                    'city': org_data.get('city', ''),
                    'state': org_data.get('state', ''),
                    'organization_type': org_data.get('organization_type', ''),
                    'total_grants': funder['total_grants'],
                    'grant_count': funder['grant_count'],
                    'avg_grant': funder['avg_grant'],
                    'latest_filing': funder['latest_filing']
                })
        
        funder_report_df = pd.DataFrame(funder_details)
        funder_report_filename = f'data/targeted_funders_{CHOSEN_FOCUS}_{timestamp}.csv'
        funder_report_df.to_csv(funder_report_filename, index=False)
        print(f"🎯 Saved targeted funders report: {funder_report_filename}")
    
    # 3. Create Excel workbook with multiple sheets
    excel_filename = f'data/comprehensive_analysis_{CHOSEN_FOCUS}_{timestamp}.xlsx'
    
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        
        # All prospects
        prospect_list.to_excel(writer, sheet_name='All_Prospects', index=False)
        
        # Donor-advised funds
        donor_funds_df.to_excel(writer, sheet_name='Donor_Advised_Funds', index=False)
        
        # Private foundations
        foundations_df.to_excel(writer, sheet_name='Private_Foundations', index=False)
        
        # Targeted funders (if available)
        if 'relevant_funders' in locals() and not relevant_funders.empty:
            relevant_funders.to_excel(writer, sheet_name='Targeted_Funders', index=False)
        
        # Grants data sample (first 1000 rows to avoid huge files)
        if not grants_df.empty:
            grants_sample = grants_df.head(1000)
            grants_sample.to_excel(writer, sheet_name='Sample_Grants', index=False)
    
    print(f"📊 Saved comprehensive Excel report: {excel_filename}")
    
    # 4. Create summary statistics
    summary_stats = {
        'analysis_date': datetime.now().isoformat(),
        'focus_area': CHOSEN_FOCUS,
        'total_prospects': len(prospect_list),
        'donor_advised_funds': len(donor_funds_df),
        'private_foundations': len(foundations_df),
        'grant_records': len(grants_df),
        'targeted_funders': len(relevant_funders) if 'relevant_funders' in locals() else 0
    }
    
    if not grants_df.empty:
        summary_stats.update({
            'total_grant_amount': float(grants_df['amount'].sum()),
            'avg_grant_amount': float(grants_df['amount'].mean()),
            'median_grant_amount': float(grants_df['amount'].median())
        })
    
    summary_filename = f'data/analysis_summary_{CHOSEN_FOCUS}_{timestamp}.json'
    with open(summary_filename, 'w') as f:
        json.dump(summary_stats, f, indent=2)
    
    print(f"📈 Saved analysis summary: {summary_filename}")
    
    # Print final summary
    print(f"\n{'='*60}")
    print(f"🎉 ANALYSIS COMPLETE - {CHOSEN_FOCUS.upper()}")
    print(f"{'='*60}")
    print(f"📊 Total prospects identified: {len(prospect_list)}")
    if 'relevant_funders' in locals():
        print(f"🎯 Targeted funders found: {len(relevant_funders)}")
    print(f"📁 Files exported to: /data")
    print(f"{'='*60}")
    
    # Data Export and Next Steps
    print("📤 Data Export & Next Steps")
    print("=" * 40)

    # Export filtered datasets for specific use cases
    export_dir = Path('exports')
    export_dir.mkdir(exist_ok=True)

    if 'all_orgs' in locals() and not all_orgs.empty:
        # 1. Export by geographic region
        print("🌍 Creating geographic exports...")
        
        # Top 5 states
        top_states = all_orgs['state'].value_counts().head(5)
        for state in top_states.index:
            state_orgs = all_orgs[all_orgs['state'] == state]
            filename = f'exports/{state.lower()}_organizations.csv'
            state_orgs.to_csv(filename, index=False)
            print(f"   ✅ {state}: {len(state_orgs)} organizations → {filename}")
        
        # 2. Export high-capacity organizations
        if 'revenue' in all_orgs.columns:
            high_capacity = all_orgs[all_orgs['revenue'] >= 1000000]  # $1M+ revenue
            if not high_capacity.empty:
                high_capacity.to_csv('exports/high_capacity_organizations.csv', index=False)
                print(f"   ✅ High-capacity orgs: {len(high_capacity)} organizations → exports/high_capacity_organizations.csv")
        
        # 3. Export by organization type
        for org_type in all_orgs['data_source'].unique():
            type_orgs = all_orgs[all_orgs['data_source'] == org_type]
            safe_name = org_type.lower().replace(' ', '_')
            filename = f'exports/{safe_name}.csv'
            type_orgs.to_csv(filename, index=False)
            print(f"   ✅ {org_type}: {len(type_orgs)} organizations → {filename}")

    # Export research resources
    if 'research_plan' in datasets:
        research_plan = datasets['research_plan']
        
        # Priority research targets
        priority_targets = research_plan[research_plan['research_priority'] == 'High']
        if not priority_targets.empty:
            priority_targets.to_csv('exports/priority_research_targets.csv', index=False)
            print(f"   ✅ Priority targets: {len(priority_targets)} organizations → exports/priority_research_targets.csv")
        
        # Geographic research plans
        top_research_states = research_plan['state'].value_counts().head(3)
        for state in top_research_states.index:
            state_research = research_plan[research_plan['state'] == state]
            filename = f'exports/{state.lower()}_research_plan.csv'
            state_research.to_csv(filename, index=False)
            print(f"   ✅ {state} research plan: {len(state_research)} targets → {filename}")

    # Create summary report
    print(f"\n📊 Creating comprehensive summary report...")
    summary_report = []

    summary_report.append("# 📊 Nonprofit Data Analysis Summary Report")
    summary_report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    summary_report.append("")

    if 'all_orgs' in locals() and not all_orgs.empty:
        summary_report.append(f"## 🏢 Organizational Data Overview")
        summary_report.append(f"- Total organizations analyzed: {len(all_orgs)}")
        summary_report.append(f"- Donor-advised funds: {len(all_orgs[all_orgs['data_source'] == 'Donor Advised Funds'])}")
        summary_report.append(f"- Private foundations: {len(all_orgs[all_orgs['data_source'] == 'Private Foundations'])}")
        summary_report.append("")
        
        # Geographic summary
        summary_report.append(f"## 🌍 Geographic Distribution")
        top_states = all_orgs['state'].value_counts().head(5)
        for i, (state, count) in enumerate(top_states.items(), 1):
            percentage = (count / len(all_orgs)) * 100
            summary_report.append(f"{i}. {state}: {count} organizations ({percentage:.1f}%)")
        summary_report.append("")

    if 'grants' in datasets and not datasets['grants'].empty:
        grants_df = datasets['grants']
        summary_report.append(f"## 💰 Grants Data Summary")
        summary_report.append(f"- Total grant records: {len(grants_df)}")
        if 'amount' in grants_df.columns:
            grant_amounts = grants_df[grants_df['amount'] > 0]['amount']
            if not grant_amounts.empty:
                summary_report.append(f"- Total grant amount: ${grant_amounts.sum():,.2f}")
                summary_report.append(f"- Average grant size: ${grant_amounts.mean():,.2f}")
        summary_report.append("")

    # Next steps recommendations
    summary_report.append(f"## 🎯 Recommended Next Steps")
    summary_report.append(f"1. **Immediate Actions:**")
    summary_report.append(f"   - Review exported geographic and high-capacity organization lists")
    summary_report.append(f"   - Start with foundations in your local area or state")
    summary_report.append(f"   - Use research plan CSV files for systematic investigation")
    summary_report.append("")
    summary_report.append(f"2. **Research Strategy:**")
    summary_report.append(f"   - Visit foundation websites for published grants lists")
    summary_report.append(f"   - Set up Google Alerts for grant announcements")
    summary_report.append(f"   - Use grants_collection_template.csv to record findings")
    summary_report.append("")
    summary_report.append(f"3. **Advanced Data Collection:**")
    summary_report.append(f"   - Consider Candid Foundation Directory subscription")
    summary_report.append(f"   - Download IRS 990 Schedule I forms for detailed grants data")
    summary_report.append(f"   - Re-run data_collector.py periodically for updates")

    # Save summary report
    with open('exports/analysis_summary_report.md', 'w') as f:
        f.write('\n'.join(summary_report))

    print(f"   ✅ Summary report → exports/analysis_summary_report.md")

    print(f"\n🎉 Export Complete!")
    print(f"📁 All exported files available in: ./exports/")
    print(f"📋 Review analysis_summary_report.md for comprehensive overview")
    print(f"\n✅ Ready for grant research and outreach activities!")
    
else:
    print("❌ No data available for export")
    print("💡 Run 'python collect_data.py' first")

## 7. Next Steps & Action Items

### 🎯 Research Phase
1. **Review Your Prospect Lists**: Start with the targeted funders who have a history of supporting your cause area
2. **Research Each Funder**: 
   - Visit their websites
   - Check recent 990 filings on ProPublica
   - Understand their application processes and deadlines
3. **Prioritize Prospects**: Focus on local funders and those with grant sizes matching your needs

### 📝 Outreach Phase
1. **Prepare Compelling Proposals** that clearly align with each funder's interests
2. **Follow Guidelines Exactly** - respect word limits, deadlines, and submission processes
3. **Build Relationships** before asking for funding when possible
4. **Track Your Outreach** and follow up appropriately

### 🔄 Continuous Improvement
1. **Re-run Analysis Quarterly** to find new funders and updated information
2. **Refine Keywords** based on successful grants and feedback
3. **Expand Geographic Search** if local options are limited
4. **Track Success Rates** to improve your approach

---

## 📚 Additional Resources

- **ProPublica Nonprofit Explorer**: https://projects.propublica.org/nonprofits/
- **Foundation Directory Online**: https://fdo.foundationcenter.org/
- **Guidestar**: https://www.guidestar.org/
- **GrantSpace**: https://grantspace.org/

---

**🎉 Congratulations!** You've successfully analyzed IRS 990 data to identify potential funders. Remember: fundraising is about building relationships. Use this data as a starting point for meaningful connections with funders who share your mission.

### 💡 Pro Tips
- **Quality over quantity**: Better to research 10 funders thoroughly than 100 superficially
- **Local focus**: Geographic proximity often increases funding likelihood  
- **Timing matters**: Many foundations have annual deadlines
- **Persistence pays**: Building relationships takes time but yields better results